# Análise de Marcha
**Bruno da Silva Cunha**

Universidade Federal do Espírio Santo

**1. Download, Crop e Gerar Keyframes**

Este notebook é responsável por realizar o download dos vídeos do youtube, fazer o crop, ou seja, pegar apenas a parte do vídeo que contém a marcha, e gerar os keyframes.

Neste notebook é importante utilizar uma instancia com GPU.

## Inicializando

In [1]:
!pip install yt_dlp
!pip install ultralytics

In [2]:
from google.colab import drive


Abaixo as constantes que serão utilizadas nesta seção

In [3]:
BASE_PATH = "/content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha"

As informações serão compartilhadas entre os Collabs pelo drive, então será necessário acessá-lo

In [4]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [5]:
# Acessar a pasta onde contém todos os arquivos
%cd {BASE_PATH}
!ls

/content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha
data  notebooks  processed  yolo11n-pose.pt


## Carregamento dos dados

In [6]:
import pandas as pd

In [7]:
# Caminho onde estão os CSVs originais
DATA_PATH = BASE_PATH + "/data"
PROCESS_FILE_PATH = DATA_PATH + "/processed_gait_data.csv"
SAMPLE_RATE = 1 # Fins de desenvolvimento, usar uma taxa de amostragem para não realizar todos os processamentos
DEBUGG_RATE = 1 # Fins de desenvolvimento, essa taxa visa salvar uma parte dos vídeos / crops

In [8]:
# Lê e concatena todos em um único DataFrame
df = pd.read_csv(PROCESS_FILE_PATH)

print(df.shape)
df.head()


(205, 13)


,seq,frame_num,cam_view,dataset,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time
0,cljvx44jt003v3n6lgox22f0q,3920,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,3920,4057,True,24.0,163.333,169.042
1,cljvx4wos003z3n6l6bdtpbfj,4269,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,4269,4362,True,24.0,177.875,181.750
2,cljvx5r6q00433n6l9t84rm4c,5011,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5011,5266,True,24.0,208.792,219.417
3,cljvx6sap00473n6la59xatps,5295,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5295,5376,True,24.0,220.625,224.000
4,cljvx7axd004b3n6lftkobhxs,5474,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5474,5566,True,24.0,228.083,231.917


In [9]:
# Pegando uma amostra dos dados caso esteja configurado
if SAMPLE_RATE > 0 and SAMPLE_RATE < 1:
  sampled_ids = df['id'].drop_duplicates().sample(frac=SAMPLE_RATE)
  df = df[df['id'].isin(sampled_ids)]
  print(sampled_ids.info())

In [10]:
  df


,seq,frame_num,cam_view,dataset,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time
0,cljvx44jt003v3n6lgox22f0q,3920,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,3920,4057,True,24.0,163.333,169.042
1,cljvx4wos003z3n6l6bdtpbfj,4269,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,4269,4362,True,24.0,177.875,181.750
2,cljvx5r6q00433n6l9t84rm4c,5011,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5011,5266,True,24.0,208.792,219.417
3,cljvx6sap00473n6la59xatps,5295,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5295,5376,True,24.0,220.625,224.000
4,cljvx7axd004b3n6lftkobhxs,5474,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5474,5566,True,24.0,228.083,231.917
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,cljxjsfhr000t3n6l1t0t0i5v,526,right side,Normal Gait,normal,VL0AOiZt_lg,https://www.youtube.com/watch?v=VL0AOiZt_lg,526,613,True,30.0,17.533,20.433
201,cljawf4e4000h3n6lri4oz9uy,1181,left side,Abnormal Gait,antalgic,4SZ0b9rjLtw,https://www.youtube.com/watch?v=4SZ0b9rjLtw,1181,1368,True,30.0,39.367,45.600
202,cljawh4c7000m3n6lkz9es9bl,1470,right side,Abnormal Gait,antalgic,4SZ0b9rjLtw,https://www.youtube.com/watch?v=4SZ0b9rjLtw,1470,1614,True,30.0,49.000,53.800
203,cljnyxctl001v3n6l9flqlh5e,113,left side,Abnormal Gait,exercise,_vpgXOzgVgg,https://www.youtube.com/watch?v=_vpgXOzgVgg,113,233,True,60.0,1.883,3.883


In [11]:
print("Quantidade de vídeos:", len(df.groupby("id")))

Quantidade de vídeos: 37


## Download dos vídeos

Nessa etapa é realizado o download dos vídeos do youtube para que sejam deitos as análises. Pode ser configurado uma porcentagem que os vídeos serão salvos para debugger.

In [12]:
from pathlib import Path
import yt_dlp
from tqdm import tqdm
from IPython.display import Video
import time




In [13]:
BASE_DIR = Path(BASE_PATH)
PROCESSED_DIR = Path(BASE_DIR / f"processed"); PROCESSED_DIR.mkdir(exist_ok=True, parents=True)
VIDEOS_DIR = Path(PROCESSED_DIR / f"videos"); VIDEOS_DIR.mkdir(exist_ok=True, parents=True)


In [14]:
def is_video_downloaded(id, path):
  path = path / f"{id}_video.mp4"
  return path.exists()

# Função que realiza o download do vídeo e salva no caminho especificado
def download_video(id, row, path):
  out = path / f"{id}_video.mp4"
  # Caso o vídeos já esteja salvo, não realiza o download novamente
  if is_video_downloaded(id, path):
    return out

  url = row['url'] if pd.notna(row.get('url')) else f"https://www.youtube.com/watch?v={id}"
  out = path / f"{id}_video.mp4"
  ydl_opts = {
    "outtmpl": str(out),
    "format": "bestvideo[height<=720][ext=mp4]",
    "merge_output_format": "mp4",
    "cookiefile": "/content/cookies.txt"
  }
  try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.extract_info(url, download=True)
  except Exception as e:
    print(f"Erro ao baixar o vídeo {id}: {e}")
    return None



In [15]:
if "download_video_success" not in df.columns:
  df["download_video_success"] = None

# Reseta a coluna download_vide_success de false para None, para reprocessar
df.loc[df["download_video_success"] == False, "download_video_success"] = None


# Percorre todo o dataframe e realiza o download dos vídeos
for id, row in tqdm(df.iterrows(), total=len(df)):
  if df.loc[df["id"] == row["id"], "download_video_success"].iloc[0] is not None:
    continue

  path = download_video(row["id"], row, VIDEOS_DIR)
  if path is None:
    df.loc[df["id"] == row["id"], "download_video_success"] = False
  else:
    df.loc[df["id"] == row["id"], "download_video_success"] = True
    df.loc[df["id"] == row["id"], "video_path"] = path
  time.sleep(1)

df.head()

100%|██████████| 205/205 [00:37<00:00,  5.51it/s]


,seq,frame_num,cam_view,dataset,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time,download_video_success,video_path
0,cljvx44jt003v3n6lgox22f0q,3920,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,3920,4057,True,24.0,163.333,169.042,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
1,cljvx4wos003z3n6l6bdtpbfj,4269,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,4269,4362,True,24.0,177.875,181.750,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
2,cljvx5r6q00433n6l9t84rm4c,5011,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5011,5266,True,24.0,208.792,219.417,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
3,cljvx6sap00473n6la59xatps,5295,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5295,5376,True,24.0,220.625,224.000,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
4,cljvx7axd004b3n6lftkobhxs,5474,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,5474,5566,True,24.0,228.083,231.917,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...


Verificar se todos os vídeos foram baixados com sucesso. Caso contrário, executar o bloco acima novamente

In [16]:
df.value_counts('download_video_success')

,count
download_video_success,
True,205


In [17]:
Video(df.iloc[0]["video_path"], embed=True, height=300)

Output hidden; open in https://colab.research.google.com to view.

## Cropar vídeos


Neste etapa será feito o corte dos vídeos para pegar apenas os momentos que ocorrem a marcha

In [18]:
import subprocess
import torch



In [19]:
import subprocess, shlex, os, tempfile
from pathlib import Path

# Garante que VIDEOS_DIR exista
VIDEOS_DIR = Path(PROCESSED_DIR / f"videos/crops"); VIDEOS_DIR.mkdir(exist_ok=True, parents=True)

def run_ffmpeg(cmd):
    # cmd é uma lista; executa e lança erro se falhar
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if proc.returncode != 0:
        raise RuntimeError(f"FFmpeg falhou:\n{proc.stderr}")
    return proc

def cut_segment(input_path, start_time, end_time, fps, use_nvenc=False):
    """Corta um trecho e retorna o caminho do arquivo temporário gerado.
       precise=True => reencode para cortes exatos
    """
    import tempfile, os, subprocess
    tmp_fd, tmp_path = tempfile.mkstemp(suffix=".mp4")
    os.close(tmp_fd)
    tmp_path = Path(tmp_path)

    # ordem: -ss antes do -i -> corte rápido, menos preciso
    cmd = [
        "ffmpeg",
        "-y",
        "-ss", str(start_time),
        "-to", str(end_time),
        "-i", str(input_path),
        "-c", "copy"
    ]

    cmd += [str(tmp_path)]

    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if proc.returncode != 0:
        raise RuntimeError(f"Erro ao cortar {input_path}:\n{proc.stderr}")
    return tmp_path


def concat_segments(segment_paths, out_path):
    """Concatena segmentos usando concat demuxer (sem recompactar áudio entre cortes)."""
    # Cria lista para o demuxer
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".txt") as f:
        list_path = f.name
        for p in segment_paths:
            # Escapa corretamente eventuais aspas simples e espaços
            safe_path = str(p).replace("'", "'\\''")
            f.write(f"file '{safe_path}'\n")

    # Para evitar erros de timebase, fazemos uma recompressão única na concatenação
    cmd = [
        "ffmpeg",
        "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", list_path,
        "-c:v", "copy",  # tenta copiar; se der problema, reencode abaixo (descomente linha)
        "-c:a", "aac", "-b:a", "128k",
        str(out_path)
    ]
    # Se copiar falhar devido a parâmetros distintos, troque "-c:v copy" por libx264/h264_nvenc como no corte.

    try:
        run_ffmpeg(cmd)
    finally:
        try:
            os.remove(list_path)
        except OSError:
            pass

def cortar_e_manter_partes(df):
    """
    Para cada (id, video_path), corta as janelas [start_time, end_time] e concatena,
    gerando um único arquivo {id}_crop.mp4 em VIDEOS_DIR.
    Atualiza df['crop_path'] para todas as linhas do grupo.
    """
    use_nvenc = torch.cuda.is_available()

    # Ordena por início para concatenar em ordem
    df_sorted = df.sort_values(["id", "video_path", "start_time"]).copy()

    for (vid, vpath, fps), group in df_sorted.groupby(["id", "video_path", "fps"], sort=False):
        input_path = Path(vpath)
        out_path = VIDEOS_DIR / f"{vid}_crop.mp4"

        if out_path.exists():
            print(f"✅ {out_path.name} já existe — pulando processamento.")
            df.loc[group.index, "crop_path"] = str(out_path)
            continue

        print(f"\nProcessando vídeo {vid} -> {input_path}")

        # 1) corta janelas
        segments = []
        try:
            for _, row in group.iterrows():
                s, e = float(row["start_time"]), float(row["end_time"])

                duration = e - s
                # descarta janelas curtas (< 1 segundo)
                if duration < 1.0:
                    print(f"  ⚠️ Janela descartada (duração {duration:.3f}s < 1s)")
                    continue

                print(f"  - Cortando janela [{s:.3f}s, {e:.3f}s]")
                seg = cut_segment(input_path, s, e, fps, use_nvenc=use_nvenc)
                segments.append(seg)

            # 2) concatena
            print(f"  - Concatenando {len(segments)} segmentos em {out_path}")
            concat_segments(segments, out_path)

            # 3) atualiza df para todas as linhas deste grupo
            idxs = group.index
            df.loc[idxs, "crop_path"] = str(out_path)

        finally:
            # Remove temporários
            for p in segments:
                try:
                    Path(p).unlink(missing_ok=True)
                except Exception:
                    pass

    return df


In [20]:
df = cortar_e_manter_partes(df)
print("Concluído.")

✅ -us6-5kgu4g_crop.mp4 já existe — pulando processamento.
✅ 0XM23zRNHM8_crop.mp4 já existe — pulando processamento.
✅ 2LbERGI34lA_crop.mp4 já existe — pulando processamento.
✅ 3FXUw98rrUY_crop.mp4 já existe — pulando processamento.
✅ 4KU6G7Iv8ZM_crop.mp4 já existe — pulando processamento.
✅ 4SZ0b9rjLtw_crop.mp4 já existe — pulando processamento.
✅ AWS_zIFfGTg_crop.mp4 já existe — pulando processamento.
✅ DfRhvdCiUJk_crop.mp4 já existe — pulando processamento.
✅ EHymg4AGMJs_crop.mp4 já existe — pulando processamento.
✅ FTHc-TJOQ34_crop.mp4 já existe — pulando processamento.
✅ FoWTST-lrZw_crop.mp4 já existe — pulando processamento.
✅ G6QVztrW93k_crop.mp4 já existe — pulando processamento.
✅ Ha9LKXZfWBQ_crop.mp4 já existe — pulando processamento.
✅ KK7Wr8-tiZY_crop.mp4 já existe — pulando processamento.
✅ KnvYRnTA3XQ_crop.mp4 já existe — pulando processamento.
✅ MN4vnaNwIsA_crop.mp4 já existe — pulando processamento.
✅ MTNB0MOO_yk_crop.mp4 já existe — pulando processamento.
✅ QO8D-comBy8_

In [21]:
unique_ids = df['id'].drop_duplicates().tolist()
for video_id in unique_ids[:1]:
    video_path = df[df['id'] == video_id]['crop_path'].iloc[0]
    print(f"Displaying video for ID: {video_id}")
    display(Video(video_path, embed=True, height=300))

Output hidden; open in https://colab.research.google.com to view.

## Gerar os Keyframes

Nesta etapa pegará todos os vídeos e será utilizado o YOLO para gerar os keyframes

In [22]:
from scipy.signal import savgol_filter
from ultralytics import YOLO
import numpy as np
import random


In [23]:
def clean_kps_seq(kps_seq):
    """
    Limpa e normaliza uma sequência de keypoints [T,17,3]
    Retorna uma nova sequência com (x,y) centralizados e normalizados.
    """
    kps = kps_seq.copy()

    # 1. Substituir NaNs por interpolação linear
    for j in range(kps.shape[1]*2):  # x e y de cada ponto
        t_series = kps[:, j//2, j%2]
        nans = np.isnan(t_series)
        if nans.any():
            idx = np.arange(len(t_series))
            if (~nans).sum() > 1:
                t_series[nans] = np.interp(idx[nans], idx[~nans], t_series[~nans])
            else:
                t_series[nans] = 0
        kps[:, j//2, j%2] = t_series

    # 2. Zerar keypoints com confiança muito baixa
    conf_mask = kps[:,:,2] < 0.2
    kps[conf_mask, :2] = np.nan

    # 3. Centralizar no quadril (média dos keypoints 11 e 12 - padrão COCO)
    left_hip, right_hip = 11, 12
    center = np.nanmean(kps[:, [left_hip, right_hip], :2], axis=1)
    kps[:, :, :2] -= center[:, np.newaxis, :]

    # 4. Normalizar pela distância entre ombros (escala corporal)
    left_sh, right_sh = 5, 6
    shoulder_dist = np.linalg.norm(kps[:, left_sh, :2] - kps[:, right_sh, :2], axis=1)
    scale = np.nanmedian(shoulder_dist)
    kps[:, :, :2] /= (scale + 1e-6)

    # 5. Suavizar as trajetórias
    for j in range(kps.shape[1]*2):
        seq = kps[:, j//2, j%2]
        if len(seq) >= 9:
            kps[:, j//2, j%2] = savgol_filter(seq, window_length=9, polyorder=2)

    return kps

# pares L<->R no COCO-17 (0=nariz, 5=ombro esq, 6=ombro dir, etc.)
LR_PAIRS = [(5,6), (7,8), (9,10), (11,12), (13,14), (15,16)]  # ombros, cotovelos, punhos, quadris, joelhos, tornozelos
LEFT_HIP, RIGHT_HIP = 11, 12  # p/ medir direção

def flip_lr_keypoints_xy(kps_xy: np.ndarray, width: float) -> np.ndarray:
    """
    kps_xy: (T,17,2) em pixels. Espelha horizontalmente e troca labels L/R.
    """
    seq = kps_xy.copy()
    # flip horizontal em x
    seq[..., 0] = width - seq[..., 0]
    # troca L <-> R
    for a, b in LR_PAIRS:
        seq[:, [a, b], :] = seq[:, [b, a], :]
    return seq

def canonicalize_direction(kps_seq: np.ndarray, width: float,
                           left_hip: int = LEFT_HIP, right_hip: int = RIGHT_HIP,
                           min_valid: int = 5) -> np.ndarray:
    """
    kps_seq: (T,17,3) ou (T,17,2). Se deslocamento do centro de quadris for negativo,
             espelha a sequência para normalizar direção →.
    width:   largura do frame em pixels.
    min_valid: nº mínimo de frames válidos para decidir direção.
    """
    has_conf = (kps_seq.shape[-1] == 3)
    xy = kps_seq[..., :2].copy()

    # centro de quadris (média L/R); ignora NaN
    hip = np.nanmean(xy[:, [left_hip, right_hip], 0], axis=1)  # (T,)
    valid = np.isfinite(hip)
    if valid.sum() < min_valid:
        # não dá para decidir — retorna como está
        return kps_seq

    # deslocamento em x do início para o fim (usando primeiros/últimos válidos)
    start_x = hip[valid][0]
    end_x   = hip[valid][-1]
    delta   = end_x - start_x

    if delta < 0:  # andou para a esquerda ⇒ espelhar
        flipped_xy = flip_lr_keypoints_xy(xy, width)
        if has_conf:
            out = kps_seq.copy()
            out[..., :2] = flipped_xy
            return out
        return flipped_xy
    return kps_seq

In [26]:
# --- config ---
CONF = 0.25
IMG_SIZE = 640
VID_STRIDE = 2          # processa 1 a cada 2 frames (mais rápido)
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

# parâmetros anti-falso-positivo
MIN_MULTI_SECONDS = 3  # tempo mínimo contínuo com >1 pessoa para descartar
DEFAULT_FPS = 30.0       # fallback se df não tiver fps

# pastas
POSES_DIR = Path(PROCESSED_DIR) / "poses"
POSES_DIR.mkdir(parents=True, exist_ok=True)

# cols obrigatórias
for col in ("multi_person", "poses_path"):
    if col not in df.columns:
        df[col] = None if col == "poses_path" else False

# modelo
model = YOLO("yolo11n-pose.pt")

# antes do loop
df_uniques = df.sort_values(by=["id"]).drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)


for idx, row in tqdm(df_uniques.iterrows(), total=len(df_uniques)):
    vid_id   = row["id"]
    src_path = Path(row.get("crop_path") or row.get("video_path"))
    out_path = POSES_DIR / f"{vid_id}_poses.npy"

    # ... (checagens suas inalteradas)

    fps = float(row["fps"]) if "fps" in df_uniques.columns else DEFAULT_FPS
    min_multi_frames = max(2, int(round(fps * MIN_MULTI_SECONDS)))
    multi_consec = 0
    multi_flag = False

    save_vis = (random.random() < DEBUGG_RATE)

    preds = model.predict(
        source=str(src_path),
        stream=True,
        conf=CONF,
        imgsz=IMG_SIZE,
        vid_stride=VID_STRIDE,
        device=DEVICE,
        verbose=False,
        save=save_vis,
        exist_ok=True,
        project=VIDEOS_DIR
    )

    kps_seq = []
    frame_w = None  # <--- largura do frame para o flip

    for r in preds:
        if frame_w is None:                      # <--- pega a largura
            try:
                frame_w = float(r.orig_shape[1]) # (h, w)
            except Exception:
                frame_w = None

        n_people = len(r.boxes) if (r.boxes is not None) else 0

        if n_people > 1:
            multi_consec += 1
            if multi_consec >= min_multi_frames:
                multi_flag = True
                print(f"❌ >1 pessoa por {multi_consec} frames (~{multi_consec/fps:.2f}s) — descartando: {vid_id}")
                break
        else:
            multi_consec = 0

        if (r.keypoints is None) or (len(r.keypoints) == 0):
            kps_seq.append(np.full((17, 3), np.nan, dtype=np.float32))
            continue

        # pessoa principal (maior bbox)
        areas = (r.boxes.xywh[:, 2] * r.boxes.xywh[:, 3]).cpu().numpy()
        i = int(areas.argmax())

        # garanta que veio em pixels (x,y,conf)
        kps = r.keypoints.data[i].cpu().numpy().astype(np.float32)  # (17,3)
        kps_seq.append(kps)

    if multi_flag:
        df_uniques.loc[df_uniques["id"] == vid_id, "multi_person"] = True
        if out_path.exists():
            try: os.remove(out_path)
            except: pass
        continue

    if len(kps_seq) == 0:
        print(f"⚠️  Sem frames válidos: {vid_id}")
        continue

    # --- limpeza + CANONICALIZAÇÃO DE DIREÇÃO ---
    kps_seq = np.asarray(kps_seq, dtype=np.float32)  # [T,17,3]
    kps_seq_clean = clean_kps_seq(kps_seq)           # sua função

    # se souber a largura, normaliza direção (→); mantém conf
    if frame_w is not None and np.isfinite(frame_w) and frame_w > 0:
        kps_seq_clean = canonicalize_direction(kps_seq_clean, width=frame_w)

    np.save(out_path, kps_seq_clean)
    df_uniques.loc[df_uniques["id"] == vid_id, "poses_path"] = str(out_path)



  0%|          | 0/37 [00:00<?, ?it/s]

Results saved to /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/predict


  3%|▎         | 1/37 [00:07<04:29,  7.48s/it]

Results saved to /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/predict


  5%|▌         | 2/37 [00:08<02:12,  3.79s/it]

...

/tmp/ipython-input-1560986133.py:26: RuntimeWarning: Mean of empty slice
  center = np.nanmean(kps[:, [left_hip, right_hip], :2], axis=1)
/tmp/ipython-input-1560986133.py:72: RuntimeWarning: Mean of empty slice
  hip = np.nanmean(xy[:, [left_hip, right_hip], 0], axis=1)  # (T,)
 95%|█████████▍| 35/37 [09:58<00:28, 14.38s/it]

❌ >1 pessoa por 90 frames (~3.00s) — descartando: vfJBx50YcDw
Results saved to /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/predict


 97%|█████████▋| 36/37 [10:02<00:11, 11.36s/it]

Results saved to /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/predict


100%|██████████| 37/37 [10:11<00:00, 16.53s/it]


In [27]:
input_pose = VIDEOS_DIR / f'predict/{df_uniques.iloc[20].id}_crop.avi'
output_pose = VIDEOS_DIR / f'predict/{df_uniques.iloc[20].id}_pose.mp4'
cmd = [
  "ffmpeg",
  "-i", input_pose,
  "-vcodec", "libx264",
  output_pose,
]
subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
Video(output_pose, embed=True, height=300)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
df_uniques[df_uniques["multi_person"] == True]

,seq,frame_num,cam_view,dataset,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time,download_video_success,video_path,crop_path,multi_person,poses_path
3,cljxlzzdt00453n6l50zvip0f,198,left side,Normal Gait,normal,3FXUw98rrUY,https://www.youtube.com/watch?v=3FXUw98rrUY,198,309,True,24.0,8.250,12.875,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None
8,cll16mmxj004a3o6lims8zdjc,5528,left side,Normal Gait,normal,EHymg4AGMJs,https://www.youtube.com/watch?v=EHymg4AGMJs,5528,5618,True,25.0,221.120,224.720,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None
14,cllenvewz00393o6lvc1dc7qr,13397,left side,Normal Gait,normal,KnvYRnTA3XQ,https://www.youtube.com/watch?v=KnvYRnTA3XQ,13397,13504,True,60.0,223.283,225.067,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None
18,clk7vxpkh004b3n6lh19y5sxu,1286,right side,Normal Gait,normal,RMdC8Pa3VbU,https://www.youtube.com/watch?v=RMdC8Pa3VbU,1286,1414,True,30.0,42.867,47.133,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None
28,cllbz3nkx001u3o6lc1n8stel,12660,right side,Normal Gait,normal,hSIYGZhRGd4,https://www.youtube.com/watch?v=hSIYGZhRGd4,12660,12725,True,30.0,422.000,424.167,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None
34,cll9vb7gq005q3o6lmoiqs9sw,30,right side,Abnormal Gait,exercise,vfJBx50YcDw,https://www.youtube.com/watch?v=vfJBx50YcDw,30,330,True,30.0,1.000,11.000,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,True,None


In [29]:
before = len(df_uniques)
cleaned = df_uniques[df_uniques["multi_person"] != True].copy()
after = len(cleaned)

print(f"🧹 Removidos {before - after} vídeos com múltiplas pessoas. Restam {after}.")

🧹 Removidos 6 vídeos com múltiplas pessoas. Restam 31.


In [30]:
cleaned

,seq,frame_num,cam_view,dataset,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time,download_video_success,video_path,crop_path,multi_person,poses_path
0,cll9usru4002x3o6lx0n272sp,592,left side,Abnormal Gait,exercise,-us6-5kgu4g,https://www.youtube.com/watch?v=-us6-5kgu4g,592,933,True,30.0,19.733,31.100,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
1,cljo1kedv003c3n6l4p52mpyp,143,left side,Abnormal Gait,exercise,0XM23zRNHM8,https://www.youtube.com/watch?v=0XM23zRNHM8,143,214,True,25.0,5.720,8.560,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
2,cljxlvp0c003e3n6lhbqb87qc,133,left side,Normal Gait,normal,2LbERGI34lA,https://www.youtube.com/watch?v=2LbERGI34lA,133,167,True,60.0,2.217,2.783,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
4,cljxm3ajs004m3n6lk7unsn04,1376,left side,Normal Gait,normal,4KU6G7Iv8ZM,https://www.youtube.com/watch?v=4KU6G7Iv8ZM,1376,1476,True,30.0,45.867,49.200,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
5,cljawh4c7000m3n6lkz9es9bl,1470,right side,Abnormal Gait,antalgic,4SZ0b9rjLtw,https://www.youtube.com/watch?v=4SZ0b9rjLtw,1470,1614,True,30.0,49.000,53.800,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
6,cljvx44jt003v3n6lgox22f0q,3920,left side,Abnormal Gait,abnormal,AWS_zIFfGTg,https://www.youtube.com/watch?v=AWS_zIFfGTg,3920,4057,True,24.0,163.333,169.042,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
7,cljwtauxc00093n6lcfjk03tn,66,left side,Normal Gait,normal,DfRhvdCiUJk,https://www.youtube.com/watch?v=DfRhvdCiUJk,66,165,True,30.0,2.200,5.500,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
9,cljxkbce1001f3n6lfsg0zbg4,1864,right side,Normal Gait,normal,FTHc-TJOQ34,https://www.youtube.com/watch?v=FTHc-TJOQ34,1864,1955,True,30.0,62.133,65.167,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
10,cljwcp4ds00233n6lr9hvtf73,388,left side,Abnormal Gait,exercise,FoWTST-lrZw,https://www.youtube.com/watch?v=FoWTST-lrZw,388,686,True,30.0,12.933,22.867,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...
11,cll1tlpxj001c3o6lkqkvvqhq,330,right side,Abnormal Gait,exercise,G6QVztrW93k,https://www.youtube.com/watch?v=G6QVztrW93k,330,460,True,30.0,11.000,15.333,True,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...,False,/content/drive/MyDrive/Estudos/IA_CD/Modelo_Tr...


In [31]:
# Caminho para salvar o arquivo processado
PROCESSED_DATA_PATH = BASE_PATH + "/data"

# Salvar o DataFrame processado em um arquivo CSV
output_file_path = os.path.join(PROCESSED_DATA_PATH, "processed_videos_data_v2.csv")
df.to_csv(output_file_path, index=False)

print(f"DataFrame salvo em: {output_file_path}")

DataFrame salvo em: /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/data/processed_videos_data_v2.csv
